In [1]:
GLOBAL_SEED = 7532

from numpy.random import seed
seed(GLOBAL_SEED)
from tensorflow import set_random_seed
set_random_seed(GLOBAL_SEED)

import numpy as np
import pandas as pd

from keras.models import Sequential, load_model
from keras.callbacks import History 
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Dense, Dropout, Conv2D, Flatten, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


Read in the data.

In [2]:
train_set = pd.read_csv('train_set_metadata.csv')
valid_set = pd.read_csv('valid_set_metadata.csv')
test_set = pd.read_csv('test_set_metadata.csv')

train_set_partition = np.load('train_set_partition_100_parts.npy')

Compute the number of parts the train set was partitioned into.

In [3]:
n_parts = len(train_set_partition) - 1
n_parts

100

## CNN Model Training from Scratch with Data Augmentation 

In [4]:
BEST_MODEL_PATH = 'best_model.hdf5'
CURRENT_MODEL_PATH = 'current_model.hdf5'

LOSS_SCORES_PATH = 'loss_scores.npy'
ACC_SCORES_PATH = 'accuracy_scores.npy'

INPUT_SHAPE = (320, 320, 3)

EARLY_STOPPING_PATIENCE = 4
LEARNING_RATE = 0.0001
N_EPOCHS = 50
BATCH_SIZE = 8

In [5]:
def create_sequential_model(input_shape):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same', 
                     input_shape=input_shape))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())

    model.add(Conv2D(48, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())  
    
    model.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
 
    model.add(Conv2D(96, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
    
    model.add(Conv2D(128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())

    model.add(Conv2D(256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dropout(rate=0.5, seed=GLOBAL_SEED))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(rate=0.5, seed=GLOBAL_SEED))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(rate=0.5, seed=GLOBAL_SEED))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [6]:
model = create_sequential_model(INPUT_SHAPE)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 320, 320, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 160, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 160, 160, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 160, 160, 48)      13872     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 80, 80, 48)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 80, 80, 48)        192       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 80, 80, 64)        27712     
__________

Train the model.

In [7]:
loss_scores, accuracy_scores = np.array([]), np.array([])
np.save(LOSS_SCORES_PATH, loss_scores)
np.save(ACC_SCORES_PATH, accuracy_scores)

In [ ]:
gender_column_position = train_set.columns.get_loc('gender')

X_valid = np.load('valid_set_hmgd_arr.npy')
y_valid = valid_set['gender'].values

batch_limit = train_set_partition[1:] - train_set_partition[:-1]
loss_scores, accuracy_scores = np.load(LOSS_SCORES_PATH), np.load(ACC_SCORES_PATH)

for epoch in range(N_EPOCHS):
    print(f'Master epoch {epoch + 1}:')
    
    for part in range(n_parts):  
        print(f'Part {part + 1}:')
    
        train_filename = 'train_set_hmgd_arr_100_parts_' + str(part + 1).zfill(3) + '.npy'
        subrange = range(train_set_partition[part], train_set_partition[part + 1])    
        X_train = np.load(train_filename)
        y_train = train_set.iloc[subrange, gender_column_position].values

        # Model initialization/loading
        if (epoch > 0) or (part > 0):
            model = load_model(CURRENT_MODEL_PATH)            
        else:
            model = create_sequential_model(X_train.shape[1:])
            adam = Adam(lr=LEARNING_RATE)
            model.compile(optimizer=adam, loss='binary_crossentropy', 
                          metrics=['binary_accuracy'])
        
        
        data_gen = ImageDataGenerator(rotation_range=20, 
                                      width_shift_range=0.2, 
                                      height_shift_range=0.2, 
                                      horizontal_flip=True)
        
        steps_per_epoch = int(batch_limit[part] / BATCH_SIZE)
    
        history = History()
        model.fit_generator(data_gen.flow(X_train, y_train, batch_size=BATCH_SIZE), 
                            steps_per_epoch=steps_per_epoch, 
                            epochs=1,
                            callbacks=[history],
                            workers=4, 
                            verbose=2)
        
        model.save(CURRENT_MODEL_PATH)
        
        #free up memory
        del X_train
        
    epoch_evaluation = model.evaluate(X_valid, valid_set['gender'].values)    
    epoch_val_loss = epoch_evaluation[0]
    epoch_val_accuracy = epoch_evaluation[1]

    print(f'\nMaster epoch {epoch + 1} validation results:\n Loss: {epoch_val_loss}, Accuracy: {epoch_val_accuracy}\n\n')
    
    # save best model (if appropriate)
    if loss_scores.size:
        if epoch_val_loss < loss_scores.min():
            model.save(BEST_MODEL_PATH)
    else:
        model.save(BEST_MODEL_PATH)
    
    loss_scores = np.append(loss_scores, epoch_val_loss)
    accuracy_scores = np.append(accuracy_scores, epoch_val_accuracy)

    np.save(LOSS_SCORES_PATH, loss_scores)
    np.save(ACC_SCORES_PATH, accuracy_scores)    
    
    # early stopping  
    if loss_scores.size:
        if len(loss_scores) - np.argmin(loss_scores) >= EARLY_STOPPING_PATIENCE:
            break

#free up memory
del X_valid          

Loss and accuracy scores.

In [5]:
loss_scores, accuracy_scores = np.load(LOSS_SCORES_PATH), np.load(ACC_SCORES_PATH)

In [6]:
loss_scores, accuracy_scores

(array([0.38889226, 0.32604104, 0.32710215, 0.30417137, 0.28552794,
        0.28671577, 0.28633918, 0.28537287, 0.28410828, 0.28386664,
        0.28120725, 0.27707354, 0.28051974, 0.28511419, 0.28015178,
        0.279805  , 0.27498961, 0.27672839, 0.27824121, 0.29120619,
        0.28168933]),
 array([0.84506705, 0.86206897, 0.86039272, 0.87715517, 0.88194444,
        0.88338123, 0.88457854, 0.88050766, 0.8829023 , 0.88242337,
        0.88314176, 0.88769157, 0.88529693, 0.88505747, 0.88864943,
        0.8855364 , 0.88984674, 0.88888889, 0.88864943, 0.88457854,
        0.88793103]))

Read in the test data and evaluate the model.

In [5]:
X_test = np.load('test_set_hmgd_arr.npy')

In [6]:
model = load_model(BEST_MODEL_PATH)
model.evaluate(X_test, test_set['gender'].values)

7621/7621 [==============================] - 27s 4ms/step


[0.28877010718994983, 0.885316887547566]

In [7]:
# free up memory
del X_test